## Getting code and data setup locally

- Execute `dev.ipynb`
- Run following commands on CL. 
    - `python main.py --data_file data/t1.pt --param double --out_dir out; python main.py --data_file data/t2.pt --param triple --out_dir out` 
- Execute `test.ipynb` and verify factors were applied correctly.

## Getting environment setup on CHTC